# Reconstruct

![](http://reconstrue.com/projects/brightfield_neurons/allen_morphology_viz.png)

"Reconstuction" is the term of art for morphological analysis that reconstructs the structure of imaged neuron. The above image set illustrates the process.

1. The MinIP is the input, the field is bright
2. The MinIP inverted for contrast with the blue and red to come
3. The reconstructed dendrites overlaid in blue
5. The reconstructed axon overlaid in red





## Multiple stained cells
In brightfield that is almost always a single neuron. Nonetheless, multiple neurons can be biocytin-stained in the same slice of brain. In such images there is usually little overlap. There is reason to believe that such multi-stained images will be processable via the same algorithms that process single objects. [[*](https://blog.tensorflow.org/2018/05/real-time-human-pose-estimation-in.html)]

![](https://1.bp.blogspot.com/-iM-M6F-UfxY/XcCnjy4DFaI/AAAAAAAAAxo/q6XrUeXedrcM6k34_XXhKzlfl0jXLlWkgCLcBGAsYHQ/s320/2b.png)


## Banding in the data
On a semi-local level there is a banding in the data. To see an example, crop in on a random cell's soma. (Here Cell Type DB cell 713686035 is used as an example.) Then using `minmax_scale()` stretch that subset of the original intensities to the full potential range of 0 -- 255. Finally, colormap to Turbo. The vertical bands alternating yellow and red are some artifact of the input images which the reconstruction algorithms will need to deal with.




In [0]:
import skimage.io
from sklearn.preprocessing import minmax_scale

# Download cell 713686035's stack minimum intensity projection (minip)
!wget --output-document=minip.tif http://reconstrue.com/projects/brightfield_neurons/demo_images/713686035.minip_original.tif
minip_filename = os.path.join(os.getcwd(), 'minip.tif')
minip = PIL.Image.open(minip_filename)

# How much to crop from each side: ((top, bottom), (left, right))
crop_box_for_713686035 = ((2000,2000),(1750,2250))
projection_orig = skimage.io.imread(minip_filename)
minip_cropped = skimage.util.crop(projection_orig, crop_box_for_713686035)
scaled = minmax_scale(minip_cropped) * 255

minip_cropped_pil = PIL.Image.fromarray(scaled)
minip_cropped_pil.show()

<img src="http://reconstrue.com/projects/brightfield_neurons/demo_images/banded_data.png" width="100%"/>


## References

Other skeletonization methods:
- [TEASAR](http://spin2013.cs.sunysb.edu/~bender/pub/PG2000_teasar.pdf)
  - seung-lab: [Python TEASAR Skeletonization for Sparse Labels](https://github.com/seung-lab/skeletonization)
  - [TEASAR: simple alg](https://github.com/scikit-image/scikit-image/issues/1621), MIT license
  - [Skeletonize.cpp](https://github.com/funkey/imageprocessing/blob/master/Skeletonize.cpp) funkey's C++, MIT
- [DeepNeuron: An Open Deep Learning Toolbox for Neuron Tracing](https://www.biorxiv.org/content/10.1101/254318v1.full.pdf)
  - Vaa3D, Allen Institute 2018
